In [22]:
import nltk
import json
import pandas as pd
from glob import glob
from konlpy.tag import Okt
from nltk.tokenize import word_tokenize
twitter = Okt()

In [23]:
caption_list=[]

files = glob('../이미지캡셔닝/*/*.json')
for filename in files:
    with open(filename, encoding='UTF8') as f:
        d = json.load(f)
        for key in d.keys():
            for i in range(len(d[str(key)])):
                caption_list.append(d[str(key)][i]['caption'])

with open("./caption.txt","w", encoding = 'UTF8') as f:
    for caption in caption_list:
        f.write(f'{caption}\n')

In [24]:
with open("./caption.txt", encoding='UTF8') as f:
    captions=f.readlines()

In [25]:
nltk.download('punkt')
stop_words = ['.','있다.','있는','이','의','를','에','을', '있', '하', '것', '들', '그', '되', 
              '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', '년', 
              '가', '한', '지', '대하', '오', '말', '일', '그렇', '위하']

[nltk_data] Downloading package punkt to /Users/yehoon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [26]:
caption = pd.DataFrame({'caption':captions})
caption['clean_caption'] = caption['caption'].str.replace("^[가-힣a-zA-Z]+$", " ")

/Users/yehoon/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [27]:
caption['clean_caption'][0]
tokenized_caption = caption['clean_caption'].apply(lambda x: x.split()) # 토큰화
tokenized_caption = tokenized_caption.apply(lambda x: [item for item in x if item not in stop_words])
result = []

In [28]:
from gensim import corpora

In [29]:
dictionary = corpora.Dictionary(tokenized_caption)
corpus = [dictionary.doc2bow(text) for text in tokenized_caption]

In [30]:
import gensim
NUM_TOPICS = 10 #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.096*"색" + 0.064*"앞에" + 0.061*"흰색" + 0.054*"빨간" + 0.035*"텔레비전" + 0.032*"검은" + 0.031*"건물의" + 0.029*"색과" + 0.028*"노란" + 0.023*"새겨진"')
(1, '0.145*"앉아" + 0.073*"나무" + 0.031*"의자에" + 0.029*"바닥에" + 0.029*"의자," + 0.027*"방" + 0.026*"의자" + 0.022*"테이블에" + 0.021*"근접" + 0.020*"남자가"')
(2, '0.159*"서" + 0.091*"옆에" + 0.039*"기대어" + 0.038*"건물" + 0.031*"꽃이" + 0.030*"밤에" + 0.029*"남자가" + 0.024*"남자" + 0.024*"벽에" + 0.023*"여자"')
(3, '0.045*"하고" + 0.034*"사람이" + 0.033*"남자가" + 0.031*"전화를" + 0.030*"창문이" + 0.028*"욕실" + 0.020*"서서" + 0.020*"많은" + 0.020*"걷고" + 0.017*"거울이"')
(4, '0.146*"벽에" + 0.076*"걸린" + 0.072*"큰" + 0.065*"걸려" + 0.052*"건물" + 0.048*"그림" + 0.033*"흰색과" + 0.032*"붙어" + 0.029*"표지판" + 0.027*"그림의"')
(5, '0.074*"무리의" + 0.058*"사람들이" + 0.051*"사진" + 0.031*"서" + 0.025*"된" + 0.024*"밖에" + 0.020*"방의" + 0.019*"텔레비전이" + 0.018*"흑백" + 0.018*"건물"')
(6, '0.112*"창문" + 0.074*"옆에" + 0.057*"금속" + 0.047*"앉아" + 0.043*"창문에" + 0.040*"큰" + 0.038*"아래에" + 0.036*"그림이" + 0.033*"그려진" + 0.018*"창문이"')
(7, '0.185*"위에" + 0.124*"놓여" + 

In [31]:
# vector = ldamodel[test_doc]
# print(vector)

In [32]:
test_doc = corpus[-1]
vector = ldamodel[test_doc]
ldamodel.save('../model/test_model.model')

In [33]:
print(vector)

[(0, 0.020006139), (1, 0.020000136), (2, 0.020002153), (3, 0.020000136), (4, 0.020000417), (5, 0.020000136), (6, 0.8199881), (7, 0.020001883), (8, 0.020000136), (9, 0.020000787)]


In [66]:
new_model = gensim.models.ldamodel.LdaModel.load("../data/museum/model_topic/test_model.model")
